In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split



C:\Users\lalit\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [65]:
train=pd.read_csv('C:\\Users\\lalit\\Downloads\\train_2km.csv')
test=pd.read_csv('C:\\Users\\lalit\\Downloads\\test_oJ.csv')

In [89]:
submission=pd.read_csv('C:\\Users\\lalit\\Downloads\\sample_submission_LnhVWA4.csv')

In [66]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [67]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [68]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [69]:
def remove_special(text):
    return re.sub(r'[^a-zA-Z0-9\s]+','',text)

In [70]:
train['tweet'] = train['tweet'].apply(remove_special)
train.head()

,id,label,tweet
0,1,0,fingerprint Pregnancy Test httpsgooglh1MfQV an...
1,2,0,Finally a transparant silicon case Thanks to ...
2,3,0,We love this Would you go talk makememories un...
3,4,0,Im wired I know Im George I was made that way ...
4,5,1,What amazing service Apple wont even talk to m...


In [71]:
#cleaning description column
def clean(text):
    stop_words = set(stopwords.words('english'))
    stop = [x.lower() for x in stop_words]
    lemma = WordNetLemmatizer()
#Converting into lower case
    text = text.lower()
    text = re.sub('\W+',' ',text).strip() #Keeping alphanumeric characters only
    #text = text.replace('user','')
#tokenize
    text_token = word_tokenize(text)
#Replacing shortcuts
    full_words = text_token
    words_alpha = [re.sub(r"\d+",'',word) for word in full_words]
    words_big = [word for word in words_alpha if len(word)>2]
    stemmed_words = [lemma.lemmatize(word) for word in words_big]
#joining list elements to string
    clean_text = " ".join(stemmed_words)
    clean_text = clean_text.replace(' ', ' ')
    clean_text = clean_text.replace(' ', ' ')
    return clean_text


In [72]:
train["tweet"] = train["tweet"].apply(lambda x: clean(x))

In [73]:
train.head()

,id,label,tweet
0,1,0,fingerprint pregnancy test httpsgooglhmfqv and...
1,2,0,finally transparant silicon case thanks uncle ...
2,3,0,love this would you talk makememories unplug r...
3,4,0,wired know george wa made that way iphone cute...
4,5,1,what amazing service apple wont even talk abou...


In [74]:
x=train['tweet']
y=train['label']

In [75]:
#Check for na
train.isna().sum()


id       0
label    0
tweet    0
dtype: int64

In [76]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [77]:
test["tweet"] = test["tweet"].apply(lambda x: clean(x))

In [78]:
bow = CountVectorizer(max_features=500)
X_trainBoW = bow.fit_transform(x)
X_test=bow.transform(test['tweet'])

In [79]:
test['tweet'] = test['tweet'].apply(remove_special)
test.head()

,id,tweet
0,7921,hate the new iphone upgrade won let download a...
1,7922,currently shitting fucking pant apple imac cas...
2,7923,like put some rom ipad that possible yes but w...
3,7924,ipod officially dead lost all picture and vide...
4,7925,been fighting itunes all night only want the m...


In [80]:
Lr = LogisticRegression()
Lr = Lr.fit(X_trainBoW,y)
#y_pred = Lr.predict(X_testBoW)


In [81]:
X_test

<1953x500 sparse matrix of type '<class 'numpy.int64'>'
	with 15604 stored elements in Compressed Sparse Row format>

In [82]:
lr=Lr.predict(X_test)
lr

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [91]:
#d=pd.DataFrame(columns=['id','label'])
submission['label']=lr
submission.to_csv('C:\\Users\\lalit\\Downloads\\sample_submission_1.csv',index=False)

In [53]:
svm = SVC()
svm = svm.fit(X_trainBoW,y)
#y_pred_svm = svm.predict(X_testBoW)

In [57]:
y_pred_svm = svm.predict(X_test)
y_pred_svm

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [92]:
#d=pd.DataFrame(columns=['id','label'])
submission['label']=y_pred_svm
submission.to_csv('C:\\Users\\lalit\\Downloads\\sample_submission_2.csv',index=False)

In [38]:
train.dtypes

id        int64
label     int64
tweet    object
dtype: object

In [39]:
test.dtypes

id        int64
tweet    object
dtype: object

In [55]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(train)
#X_test_tfidf = tfidf.transform(X_test)
